# Predict Citations with Word to Watch Vectors
Here we try to predict whether a sentence needs a 'citation needed' tag or not based on
1) Word to watch vectors only (english, all languages)
2) Word to watch vectors + main sec indicator


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from random import shuffle
import numpy as np
import hashlib

DEPTH_SEARCH=[10,30, 50, 100, 200]
NTREES_SEARCH=[10,30, 50, 100, 200]
TEST_SIZE=0.33

In [9]:
filenames={}
filenames_raw={}
filenames_ww={}
"""
to use this, you will need: 
1) features from https://drive.google.com/open?id=1JZu67psmj2Eou2-8wQEJk4kAQfg8GDs2, to be placed in ../fastText_multilingual/features
"""
languages=['en','fr','it']
language_extended=['english','french','italian']
feadir='../fastText_multilingual/features/'
rawdir='../data_clean/'

def load_languages():
    for lan,lext in zip(languages,language_extended):
        filenames[lan]=feadir+lan+'.tsv' #files with vectors
        filenames_ww[lan]=feadir+'ww_'+lan+'.tsv' #files with vectors
        filenames_raw[lan]=rawdir+lext+'.tsv' #files with raw text

def count_negatives(negatives,positives):
    """
    for balanced data, we need to know how many negatives are out there
    """
    proportion={}
    allneg=0
    for lan in languages:
        proportion[lan]=len(negatives[lan])/float(len(negatives[lan])+len(positives[lan]))
        allneg+=len(negatives[lan])
    print 'proportion of negatives per language'
    print proportion
    return allneg

def get_values_for_crossvalidation(positives,negatives,features):
    """
    positives: list of positives
    negatives: list of negatives
    features: list of feature dictionaries, per type
    """
    values=[]
    y=[]
    ids=[]
    for lan in languages:
        shuffle(positives[lan])
        alldata=set(negatives[lan]+positives[lan][:len(negatives[lan])])
        ids=ids+list(alldata)
        for id in alldata:
            v=[]
            for f in features: #for every type of feature
                if isinstance(f[id], int):
                    v.append(f[id])
                else:
                    for element in f[id]: #append element of feature
                        v.append(element)
            values.append(np.nan_to_num(np.asarray(v)))
            y.append(labels[id])          
    #reshuffle everything for cross_validaton
    ind=range(len(y))
    shuffle(ind)
    y2=[y[i] for i in ind]
    values2=[values[i] for i in ind]
    ids2=[ids[i] for i in ind]
    return y2,values2,ids2

def perform_gridsearch_withRFC(values,y):
    """
    values: list of feature vectors
    y: labels
    returns
    max_ind: depth and estimator values
    max_val: crossval prediction accuracy
    scores: all-scores for each combination of depth and nestimators
    """
    scores={}
    #performs cross_validation in all combiantions
    for d in DEPTH_SEARCH:
        for n in NTREES_SEARCH:
            clf = RandomForestClassifier(max_depth=d, n_estimators=n)
            s = cross_val_score(clf, values, y)
            print s
            scores[str(d)+' '+str(n)]=np.mean(s)
    #computes best combination of parameters
    max_ind=''
    max_val=0
    for s in scores:
        if scores[s]>max_val:
            max_val=scores[s]
            max_ind=s
    print max_ind
    print max_val
    return max_ind,max_val,scores

def train_test_final(val_train,val_test,y_train,d,n):
    """
    just using a Random Forestc classifier on a train/test split for deployment 
    returns model and probability on the test set
    """
    clf = RandomForestClassifier(max_depth=d, n_estimators=n)
    clf.fit(val_train,y_train)
    prob=clf.predict_proba(val_test)
    return clf,prob


In [22]:

def print_top_bottom_sentences(prob,ids_test,y_test,text,labels):
    """
    here we are displaying the 
    """
    pos_proba=(np.asarray(prob).T)[1]
    indexes=np.argsort(-np.asarray(pos_proba))
    for i in indexes[:10]:
        print text[ids_test[i]]
        print y_test[i]
        print labels[ids_test[i]]#checking
    print ('********************************')
    for i in indexes[-40:]:
        print text[ids_test[i]]
        print y_test[i]
        print pos_proba[i]
        print labels[ids_test[i]]#checking

Let's load labels and vectors.. 


In [10]:
load_languages()

Now we load all data for prediction into different variables

In [15]:
"""
raw header is:
entity_id	revision_id	timestamp entity_title	section	start	offset	statement label
feature header is:
entity_id	revision_id	timestamp entity_title	section	start	offset	 label feature
"""
labels={} #whether it needs a citation or not
vectors={} #the word vectors aligned to english
main={} #is it the main section?
language={} #which language is the article from
pages={} #length of the page
start={} #starting point of the statement in the page
pagelength={} #page length, this is for future use, if we want to track where the statement is placed in the page
positives={}#statements with citation
negatives={}#statements without citation
text={}#raw text
ww={}
for lan in languages:
    positives[lan]=[] #stores the statements needing a citation
    negatives[lan]=[] #stores the statements without a citation (much less than the positives)
    fraw=open(filenames_raw[lan]) #each line in fraw correspond to the line in f
    fww=open(filenames_ww[lan]) #each line in fww correspond to the line in f
    #for each line in the vector file, record various parameters and then store the corresponding raw text with the same identifier
    with open(filenames[lan]) as f:
        for line in f:
            unique=hashlib.sha224(line).hexdigest() #unique identifier of this line
            #first, we store the raw statement text from the raw file
            lineraw=fraw.readline() #line with raw text
            rowraw=lineraw[:-1].split('\t')
            text[unique]=rowraw[-2] #where the text is placed in the line
            lineww=fww.readline() #line with raw text
            rowww=lineww[:-1].split('\t')
            ww[unique]=[float(r) for r in rowww[-1].split(',')]#where the text is placed in the line
            #now, we can get features
            row=line.split('\t')
            labels[unique]=int(row[-2])#where the label sits in the feature file
            #first append to lists of positives and negatives depending on the label
            if labels[unique]==1:
                positives[lan].append(unique)
            else:
                negatives[lan].append(unique)
            #store features
            vectors[unique]=[float(r) for r in row[-1].split(',')]
            main[unique]= 1 if row[4]=='MAIN_SECTION'else 0
            language[unique]=lan
            pages[unique]=int(row[0])
            beginning=int(row[5])
            offset=int(row[6])
            l=beginning+offset
            try:
                base=pagelength[row[0]]
                pagelength[row[0]]=l if l>base else base
            except:
                pagelength[row[0]]=l
            start[unique]=beginning


In [16]:
allneg=count_negatives(negatives,positives)

proportion of negatives per language
{'fr': 0.06976161231293711, 'en': 0.05151972709224618, 'it': 0.1304069891327509}


We now select the data for training: all negatives + an equal number of positives, using only feature vectors

In [17]:
y,values,ids=get_values_for_crossvalidation(positives,negatives,[ww])

We now run a grid search to find the good random forest parameter

In [18]:
max_ind,max_val,scores=perform_gridsearch_withRFC(values,y)

[ 0.63973328  0.6446362   0.63522259]
[ 0.65601098  0.66640518  0.64071387]
[ 0.65699157  0.66581683  0.64365562]
[ 0.65640322  0.66699353  0.64640126]
[ 0.66581683  0.67385762  0.65542263]
[ 0.62816238  0.6307119   0.62325946]
[ 0.65404981  0.64914689  0.637576  ]
[ 0.66660129  0.6597372   0.65699157]
[ 0.67699549  0.66503236  0.65914885]
[ 0.66993528  0.67640714  0.66660129]
[ 0.62424005  0.62188664  0.60992351]
[ 0.65110806  0.6573838   0.63836046]
[ 0.66464013  0.66228672  0.64895077]
[ 0.67326927  0.67503432  0.66326731]
[ 0.67660326  0.6840557   0.66954305]
[ 0.62031771  0.6295352   0.62012159]
[ 0.64542067  0.64679349  0.64483232]
[ 0.66032555  0.66424789  0.65993332]
[ 0.67385762  0.67719161  0.6573838 ]
[ 0.6760149   0.67954501  0.67170033]
[ 0.6248284   0.62149441  0.61502255]
[ 0.65444205  0.65914885  0.65208864]
[ 0.66169837  0.66307119  0.64836242]
[ 0.67699549  0.67738772  0.66365954]
[ 0.67895666  0.68131006  0.66973916]
50 200
0.676734000131


In [19]:
print ('ww+main')
y,values,ids=get_values_for_crossvalidation(positives,negatives,[ww,main])
max_ind,max_val,scores=perform_gridsearch_withRFC(values,y)
print ('ww+vecs')
y,values,ids=get_values_for_crossvalidation(positives,negatives,[ww,vectors])
max_ind,max_val,scores=perform_gridsearch_withRFC(values,y)
print ('ww+vecs+main')
y,values,ids=get_values_for_crossvalidation(positives,negatives,[ww,vectors,main])
max_ind,max_val,scores=perform_gridsearch_withRFC(values,y)

ww+main
[ 0.73087485  0.71930169  0.71537858]
[ 0.7416634   0.72852099  0.72459788]
[ 0.74695959  0.73656336  0.73048254]
[ 0.74323264  0.73617105  0.72969792]
[ 0.74597882  0.739898    0.73107101]
[ 0.69497842  0.69949     0.69733229]
[ 0.73420949  0.72832483  0.72224402]
[ 0.74578266  0.7434288   0.73283641]
[ 0.7508827   0.74382111  0.73852491]
[ 0.75735583  0.75166732  0.73970184]
[ 0.69713613  0.68987838  0.69752844]
[ 0.73519027  0.73715182  0.72479404]
[ 0.74068262  0.73479796  0.72597097]
[ 0.75166732  0.74754806  0.73401334]
[ 0.7583366   0.75107885  0.73891722]
[ 0.69713613  0.69478227  0.69027069]
[ 0.73754413  0.73420949  0.72754021]
[ 0.73950569  0.739898    0.72754021]
[ 0.74990192  0.74892115  0.73420949]
[ 0.75402118  0.74617497  0.73852491]
[ 0.71067085  0.69929384  0.69007454]
[ 0.73577874  0.72655944  0.71596705]
[ 0.74990192  0.74617497  0.72812868]
[ 0.75774814  0.74872499  0.73617105]
[ 0.75323656  0.7508827   0.73950569]
30 200
0.749574996731
ww+vecs
[ 0.65797215

In [20]:
val_train, val_test, y_train, y_test, ids_train, ids_test = train_test_split(values, y, ids, test_size=TEST_SIZE, random_state=42)

In [23]:
#clf,prob=train_test_final(val_train,val_test,y_train,50,200)
print_top_bottom_sentences(prob,ids_test,y_test,text,labels)

A week later, on 1 February, the committee met and decided the songs were innocuous, although Formby was told that he would have to get further clearance if the lyrics were changed. Bret opines that he had been the victim of a plot by a member of the Variety Artists' Federation, following Formby's scathing comments on entertainers who were too scared to leave London to entertain the troops
1
1
Mr Rhodes has declared [the innings over]." However, all the captains to whom biographer Sidney Rogerson spoke mentioned their good relations with Rhodes, although they agreed that he could be difficult. Beverley Lyon, who captained Gloucestershire between 1929 and 1934, criticised Rhodes for insisting young professionals take the game seriously, thus promoting a no-risk attitude.
1
1
Windle had trouble in pacing himself to conserve energy for later events. Talbot lamented to "never [being] astute enough to realise that he [Windle] didn't have the subtleties of pace that would have added to his a

We now lok at the effect of adding the 'main' features, i.e. a feature = 1 if the sentence is in the main section

In [11]:
y_m,values_m,ids_m=get_values_for_crossvalidation(positives,negatives,[vectors,main])

In [12]:
max_ind,max_val,scores=perform_gridsearch_withRFC(values_m,y_m)

[ 0.65954109  0.69052755  0.670459  ]
[ 0.68248676  0.71680722  0.69203609]
[ 0.69562659  0.7050402   0.72302864]
[ 0.69994116  0.72680918  0.70929776]
[ 0.70798196  0.72504413  0.72499019]
[ 0.7050402   0.72327907  0.71734013]
[ 0.65797215  0.66817023  0.67948215]
[ 0.69464601  0.71484605  0.69615536]
[ 0.73386939  0.74073348  0.73891722]
[ 0.72837811  0.75093156  0.75107885]
[ 0.74014513  0.75583448  0.75284425]
[ 0.74740145  0.75406943  0.75304041]
[ 0.67346539  0.64561679  0.65515889]
[ 0.68385958  0.69346931  0.66888976]
[ 0.71876839  0.72955481  0.72812868]
[ 0.73583056  0.74838204  0.74440957]
[ 0.75053932  0.76407139  0.75676736]
[ 0.758384    0.76250245  0.7618674 ]
[ 0.66424789  0.65601098  0.66928207]
[ 0.68425181  0.70111787  0.68340526]
[ 0.72524024  0.73739949  0.73675951]
[ 0.73641891  0.73661502  0.75205963]
[ 0.75544224  0.75583448  0.7565712 ]
[ 0.75975682  0.76564032  0.76579051]
[ 0.6609139   0.69150814  0.66555512]
[ 0.67699549  0.69111591  0.69556689]
[ 0.72680918

In [13]:
val_train, val_test, y_train, y_test, ids_train, ids_test = train_test_split(values, y, ids, test_size=TEST_SIZE, random_state=42)

In [14]:
clf,prob=train_test_final(val_train,val_test,y_train,100,200)
print_top_bottom_sentences(prob,ids_test,y_test,text,labels)

Singleton, Brian (2004). Oscar Asche, Orientalism, and British Musical Comedy. Westport, CT: Greenwood Publishing Group. p.75. ISBN978-0-275-97929-4.
0
0
The Plainsman (1936) and John Ford's Stagecoach (1939).
1
1
By 1924, many UFA Members of the Legislative Assembly (MLAs) wanted to see Greenfield leave office, both because they were frustrated with his failings and because they thought it likely that a Greenfield-led government would be defeated in the next election. Their first attempt to replace him failed when Brownlee, their intended replacement, refused to have anything to do with the plan, but a second attempt, in 1925, was successful when Brownlee agreed to take office if Greenfield personally requested that he do so. Greenfield had not wanted the job in the first place, and agreed to resign in Brownlee's favour.
0
0
He considered himself vindicated when a move to override his veto failed in the Senate the following day. Johnson believed that the Radicals would now be isolated